<a href="https://colab.research.google.com/github/saaaady/Big-Data-Heart-Disease-Prediction/blob/main/Big_Data_Project_Heart_Disease_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Heart Disease Prediction - Big Data**

## **Creating a Spark Session and reading the CSVs**

Two .csv files will be imported from the Github repository.
The data will be cleaned and then the model will be trained/tested on
"Cardiac Health Dataset.csv".
Then the model will be tested on the "FIC Heart Conditions Dataset.csv" dataset.

In [185]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import VectorAssembler

In [186]:
spark=SparkSession.builder.appName("Project").getOrCreate()

In [187]:
! git clone https://github.com/saaaady/Big-Data-Heart-Disease-Prediction

Cloning into 'Big-Data-Heart-Disease-Prediction'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 17 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 18.67 KiB | 2.33 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [188]:
ls

 Big-Data-Heart-Disease-Prediction/          'FIC Heart Conditions Dataset (Test).csv'
'Cardiac Health Dataset (Train & Test).csv'


In [189]:
cd Big-Data-Heart-Disease-Prediction/

/content/Big-Data-Heart-Disease-Prediction/Big-Data-Heart-Disease-Prediction


In [190]:
ls

'Cardiac Health Dataset (Train & Test).csv'  'FIC Heart Conditions Dataset (Test).csv'


In [191]:
data=spark.read.csv('/content/Big-Data-Heart-Disease-Prediction/Cardiac Health Dataset (Train & Test).csv',header=True,inferSchema=True)
ficdata=spark.read.csv('/content/Big-Data-Heart-Disease-Prediction/FIC Heart Conditions Dataset (Test).csv',header=True,inferSchema=True)

In [ ]:
data.show()

In [ ]:
ficdata.show()

## **Cleaning the Raw datasets**
The datasets have incorrectly labeled columns and missing data/Null value entries. The FIC dataset has extra columns which we will not be using as features which will be trimmed.

**Cardiac Health Dataset Cleaning**

In [72]:
data.show()

+---+---+---------------+---+-----------+------------+-----------+------+---------------+-------------+-----------+-----------------------+--------+-------------+
|Age|Sex|Chest pain type| BP|Cholesterol|FBS over 120|EKG results|Max HR|Exercise angina|ST depression|Slope of ST|Number of vessels fluro|Thallium|Heart Disease|
+---+---+---------------+---+-----------+------------+-----------+------+---------------+-------------+-----------+-----------------------+--------+-------------+
| 70|  1|              4|130|        322|           0|          2|   109|              0|          2.4|          2|                      3|       3|     Presence|
| 67|  0|              3|115|        564|           0|          2|   160|              0|          1.6|          2|                      0|       7|      Absence|
| 57|  1|              2|124|        261|           0|          0|   141|              0|          0.3|          1|                      0|       7|     Presence|
| 64|  1|             

In [194]:
data=data.drop('Chest pain type')
data=data.withColumnRenamed('BP','Blood Pressure(mm Hg)').withColumnRenamed('Cholesterol','Cholesterol(mg/dl)').withColumnRenamed('FBS over 120','Fasting Blood Sugar > 120(mg/dl)').withColumnRenamed('EKG results','Resting EKG').withColumnRenamed('Max HR','Max Resting Heartrate').withColumnRenamed('Exercise angina','Exercise Angina')
data = data.replace("Absence", "0", "Heart Disease")
data = data.replace("Presence", "1", "Heart Disease")
data=data.withColumn("Heart Disease", data["Heart Disease"].cast(IntegerType()))

1. Chestpain type column was dropped
2. Columns were renamed to provide clarity
3. Presence/Absence of Heart Disease denoted by 1/0 respectively.
4. Heart Disease column entries changed to integer type

In [ ]:
data.show()

**FIC Heart Conditions Dataset**

In [ ]:
ficdata.show()

In [197]:
ficdata=ficdata.drop('Age.Group','Locality  ','Others ','Marital status                       ','Family.History','Life.Style                                                                              ','Sleep','Category','Depression','Hyperlipi','Smoking','F.History','Diabetes','HTN','Allergies','Thrombolysis','BGR','B.Urea','S.Cr','S.Sodium','S.Potassium','S.Chloride','C.P.K','CK.MB','ESR','WBC','RBC','M.C.H','Hemoglobin','P.C.V','M.C.V','M.C.','M.C.H.C','PLATELET_COUNT','NEUTROPHIL','LYMPHO','MONOCYTE','EOSINO','        Others','CO','Diagnosis','Hypersensitivity','cp','trestbps','num','SK','SK.React','Reaction','Mortality','Follow.Up')
ficdata=ficdata.withColumnRenamed('BP','Blood Pressure(mm Hg)').withColumnRenamed('Gender','Sex').withColumnRenamed('chol','Cholesterol(mg/dl)').withColumnRenamed('fbs','Fasting Blood Sugar > 120(mg/dl)').withColumnRenamed('restecg','Resting EKG').withColumnRenamed('thalach','Max Resting Heartrate').withColumnRenamed('exang','Exercise Angina').withColumnRenamed('oldpeak','ST depression').withColumnRenamed('slope','Slope of ST').withColumnRenamed('ca','Number of vessels fluro').withColumnRenamed('thal','Thallium')
ficdata=ficdata.replace("Female", "0", "Sex")
ficdata=ficdata.replace("Male", "1", "Sex")
ficdata=ficdata.withColumn("Sex", ficdata["Sex"].cast(IntegerType()))
ficdata=ficdata.withColumn("Blood Pressure(mm Hg)", ficdata["Blood Pressure(mm Hg)"].cast(IntegerType()))

1. Some columns dropped
2. Columns were renamed to provide clarity
3. Male/Female o denoted by 1/0 respectively.
4. Blood Pressure(mm Hg) column entries changed to integer type

In [198]:
ficdata.show()

+---+---+---------------------+------------------+--------------------------------+-----------+---------------------+---------------+-------------+-----------+-----------------------+--------+
|Age|Sex|Blood Pressure(mm Hg)|Cholesterol(mg/dl)|Fasting Blood Sugar > 120(mg/dl)|Resting EKG|Max Resting Heartrate|Exercise Angina|ST depression|Slope of ST|Number of vessels fluro|Thallium|
+---+---+---------------------+------------------+--------------------------------+-----------+---------------------+---------------+-------------+-----------+-----------------------+--------+
| 45|  0|                  100|               341|                               1|          2|                  136|              1|          3.0|          2|                      0|       7|
| 51|  0|                   90|               305|                               0|          0|                  142|              1|          1.2|          2|                      0|       7|
| 55|  0|                  100|    

In [121]:
data.describe().show()

+-------+-----------------+------------------+---------------------+------------------+--------------------------------+------------------+---------------------+------------------+------------------+------------------+-----------------------+------------------+------------------+
|summary|              Age|               Sex|Blood Pressure(mm Hg)|Cholesterol(mg/dl)|Fasting Blood Sugar > 120(mg/dl)|       Resting EKG|Max Resting Heartrate|   Exercise Angina|     ST depression|       Slope of ST|Number of vessels fluro|          Thallium|     Heart Disease|
+-------+-----------------+------------------+---------------------+------------------+--------------------------------+------------------+---------------------+------------------+------------------+------------------+-----------------------+------------------+------------------+
|  count|              270|               270|                  270|               270|                             270|               270|                  

In [120]:
ficdata.describe().show()

+-------+-----------------+------------------+---------------------+------------------+--------------------------------+------------------+---------------------+------------------+------------------+------------------+-----------------------+------------------+
|summary|              Age|               Sex|Blood Pressure(mm Hg)|Cholesterol(mg/dl)|Fasting Blood Sugar > 120(mg/dl)|       Resting EKG|Max Resting Heartrate|   Exercise Angina|     ST depression|       Slope of ST|Number of vessels fluro|          Thallium|
+-------+-----------------+------------------+---------------------+------------------+--------------------------------+------------------+---------------------+------------------+------------------+------------------+-----------------------+------------------+
|  count|              368|               368|                  368|               368|                             368|               368|                  368|               368|               368|               

## **Feature Selection**

In [141]:
for i in data.columns:
  print("{} and Heart Disease = {}".format(i,data.stat.corr("Heart Disease",i)))

Age and Heart Disease = 0.2123221874434284
Sex and Heart Disease = 0.29772075572408496
Blood Pressure(mm Hg) and Heart Disease = 0.1553826561757688
Cholesterol(mg/dl) and Heart Disease = 0.11802053060517001
Fasting Blood Sugar > 120(mg/dl) and Heart Disease = -0.016318834144205503
Resting EKG and Heart Disease = 0.18209075568278285
Max Resting Heartrate and Heart Disease = -0.4185139653265933
Exercise Angina and Heart Disease = 0.4193027091902966
ST depression and Heart Disease = 0.4179674372274274
Slope of ST and Heart Disease = 0.33761595723299065
Number of vessels fluro and Heart Disease = 0.45533645047270893
Thallium and Heart Disease = 0.5250203329618742
Heart Disease and Heart Disease = 1.0


We will be keeping all the columns as features as no column is completely redundant in dictating presence/absence of heart disease. This is evident from the correlation between each feature and the Diagnosis.

##  **Creating a Feature Vector Column using Vector Assembler**

Merging multiple columns to a vector column.

In [142]:
data.columns

['Age',
 'Sex',
 'Blood Pressure(mm Hg)',
 'Cholesterol(mg/dl)',
 'Fasting Blood Sugar > 120(mg/dl)',
 'Resting EKG',
 'Max Resting Heartrate',
 'Exercise Angina',
 'ST depression',
 'Slope of ST',
 'Number of vessels fluro',
 'Thallium',
 'Heart Disease']

In [148]:
featuremaker=VectorAssembler(inputCols=['Age','Sex','Blood Pressure(mm Hg)','Cholesterol(mg/dl)','Fasting Blood Sugar > 120(mg/dl)','Resting EKG','Max Resting Heartrate','Exercise Angina','ST depression','Slope of ST','Number of vessels fluro','Thallium'],outputCol='Features as Vectors')

In [149]:
readyforml=featuremaker.transform(data)

In [200]:
readyforml.select('Features as Vector').show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Features as Vector` cannot be resolved. Did you mean one of the following? [`Features as Vectors`, `Heart Disease`, `ST depression`, `Resting EKG`, `Slope of ST`].;
'Project ['Features as Vector]
+- Project [Age#5489, Sex#5490, Blood Pressure(mm Hg)#6112, Cholesterol(mg/dl)#6126, Fasting Blood Sugar > 120(mg/dl)#6140, Resting EKG#6154, Max Resting Heartrate#6168, Exercise Angina#8815, ST depression#5498, Slope of ST#5499, Number of vessels fluro#5500, Thallium#5501, Heart Disease#8881, UDF(struct(Age_double_VectorAssembler_6bfc1e4ca4ee, cast(Age#5489 as double), Sex_double_VectorAssembler_6bfc1e4ca4ee, cast(Sex#5490 as double), Blood Pressure(mm Hg)_double_VectorAssembler_6bfc1e4ca4ee, cast(Blood Pressure(mm Hg)#6112 as double), Cholesterol(mg/dl)_double_VectorAssembler_6bfc1e4ca4ee, cast(Cholesterol(mg/dl)#6126 as double), Fasting Blood Sugar > 120(mg/dl)_double_VectorAssembler_6bfc1e4ca4e

The above column has been generated from the dataframe and called "Features as vectors" (Last column) which contains data points from all the columns for each individual.

## **Importing and Training Machine Learning Model**
As this is a classification problem i.e a person has Heart Disease or not, we will be using the Logistic regression model.

In [151]:
from pyspark.ml.classification import LogisticRegression

In [173]:
mldata=readyforml.select('Features as Vectors','Heart Disease')

We will train the model on a part of the "Cardiac Health Dataset" and test it on the remaining part. Then we will test the model on the FIC Heart Condtions" Dataset.

In [175]:
trainingdata,testingdata=mldata.randomSplit([0.8,0.2])

In [178]:
heartmodel=LogisticRegression(labelCol='Heart Disease',featuresCol="Features as Vectors")

In [179]:
hdp=heartmodel.fit(trainingdata)

In [180]:
summary=hdp.summary

In [182]:
summary.predictions.describe().show()

+-------+-------------------+------------------+
|summary|      Heart Disease|        prediction|
+-------+-------------------+------------------+
|  count|                209|               209|
|   mean|0.40669856459330145|0.3875598086124402|
| stddev|0.49239710956457183|0.4883629221813441|
|    min|                0.0|               0.0|
|    max|                1.0|               1.0|
+-------+-------------------+------------------+



Testing Model on Test Part of Cardiac Health Dataset